# Introduction
This notebook will attempt to adapt an existing CNN model to use LoRA for finetuning. 

Model choice - [MobileNetV2 trained on ImageNet1k](https://pytorch.org/vision/stable/models/generated/torchvision.models.mobilenet_v2.html#torchvision.models.MobileNet_V2_Weights) Reason: smallest classification CNN available via torchvision (by number of parameters), which means I can fine tune relatively faster, and there shouldn't be anything fundementally different with larger models.

Original Dataset - ImageNet1k_V2

Finetuning Dataset - [FGVCAIRCRAFT](https://pytorch.org/vision/stable/generated/torchvision.datasets.FGVCAircraft.html#torchvision.datasets.FGVCAircraft) (The dataset contains 10,000 images of aircrafts across 30 different manufacturers) Reason: Relatively small dataset (thus faster training and more representative of realworld finetuning task) and looking at airplanes is fun!

Ideas: try varying the amount of training data? how well does finetuning work on small data?

In [1]:
import lightning as L

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import torchvision
from torchvision.datasets import FGVCAircraft

from torchmetrics import Accuracy

from torchsummary import summary

import loralib as lora

import os

# make results reproducible
L.seed_everything(42)

Global seed set to 42


42

In [2]:
PATH_DATASETS = '/home/sunil/cnn-lora/data'

In [3]:
class LitCNNLora(L.LightningModule):
    
    def __init__(self, data_dir=PATH_DATASETS):
        '''
        :data_dir: - string path to where to store data
        '''
        super().__init__()

        # some hyperparameters
        self.lr = 1e-3
        self.batch_size = 24
        
        # dataset specific information
        self.data_dir = data_dir
        self.num_classes = 30 # there are 30 manufacturers in our dataset 
        
        # define lora hyper params
        lora_rank = 8

        # define the model
        self.model = torchvision.models.mobilenet_v2()
        
        # setup model for lora 
        # replace all conv layers with lora conv layers
        for name, module in self.model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                
                in_channels = module.in_channels
                out_channels = module.out_channels
                kernel_size = module.kernel_size[0]
                stride = module.stride
                padding = module.padding
                dialtion = module.dilation
                groups = module.groups
                bias = module.bias if module.bias is not None else False
                padding_mode = module.padding_mode

                new_conv = lora.Conv2d(in_channels=in_channels, 
                                     out_channels=out_channels,
                                     kernel_size=kernel_size,
                                     stride=stride,
                                     padding=padding,
                                     dilation=dialtion,
                                     groups=groups,
                                     bias=bias,
                                     padding_mode=padding_mode,
                                     r=lora_rank
                                     )


                # `parts` is essentially a list of keys we can use to determine where this layer belongs
                parts = name.split('.')
                self.set_model_feature(parts, new_conv)

            # setup layer freezes properly for lora
            # TODO is this freezing our new mlp head too?
            lora.mark_only_lora_as_trainable(self.model)

            # reset the mlp head
            self.model.classifier[1] = nn.Linear(1280, self.num_classes)
            nn.init.normal_(self.model.classifier[1].weight, 0, 0.01)
            nn.init.zeros_(self.model.classifier[1].bias)
            
            # load in pretrained weights (this excludes the classifier weights)
            self.model.load_state_dict(torch.load(self.get_initial_state_dict_path()), strict=False)


        # set up the data transform 
        self.transform = torchvision.models.mobilenetv2.MobileNet_V2_Weights.IMAGENET1K_V1.transforms
        
        # define metrics
        self.val_accuracy = Accuracy(task="multiclass", num_classes=self.num_classes)
        self.test_accuracy = Accuracy(task="multiclass", num_classes=self.num_classes)
        
    # helper function to update layers with lora layers during init
    def set_model_feature(self, keys, value):
        pointer = self.model
        while len(keys) > 1:
            key = keys.pop(0)
            pointer = getattr(pointer, key)

        key = keys.pop(0)
        pointer[int(key)] = value

    def get_initial_state_dict_path(self):
        '''
        saves the imagenet 1k weights to a state dict on disk (if necessary) and returns path to where they are saved
        '''
        weights_path = 'mobilenet_v2_imagenet1k_state_dict.pt'

        if not os.path.isfile(weights_path):
            m = torchvision.models.mobilenet_v2(weights=torchvision.models.mobilenetv2.MobileNet_V2_Weights.DEFAULT)
            state_dict = m.state_dict()
            del state_dict['classifier.1.bias']
            del state_dict['classifier.1.weight']
            torch.save(state_dict, weights_path)

        return weights_path
        
    
    def forward(self, x):
        # TODO: any transforms?
        x = self.model(x)
        return F.log_softmax(x,1)

    # centralize stuff we need in train/val/test
    def common_step(self, batch, batch_idx):
        x,y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return x, y, logits, loss

    def training_step(self, batch, batch_idx):
        _, _, _, loss = self.common_step(batch, batch_idx)
        self.log("train_loss", loss, batch_size=self.batch_size, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        _, y, logits, loss = self.common_step(batch, batch_idx)
        preds = torch.argmax(logits, dim=1)
        self.val_accuracy.update(preds, y)

        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.val_accuracy, prog_bar=True)
    
    def test_step(self, batch, batch_idx):
        _, y, logits, loss = self.common_step(batch, batch_idx)
        preds = torch.argmax(logits, dim=1)
        self.test_accuracy.update(preds, y)

        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", self.test_accuracy, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        return optimizer


    def prepare_data(self):
        # download data
        FGVCAircraft(self.data_dir, split='train', annotation_level='manufacturer', download=True) 
        FGVCAircraft(self.data_dir, split='val', annotation_level='manufacturer', download=True,)
        FGVCAircraft(self.data_dir, split='test', annotation_level='manufacturer', download=True )
        
    def setup(self, stage=None):
            self.aircraft_train = FGVCAircraft(self.data_dir, split='train', annotation_level='manufacturer', transform=self.transform())
            self.aircraft_val = FGVCAircraft(self.data_dir, split='val', annotation_level='manufacturer', transform=self.transform())
            self.aircraft_test = FGVCAircraft(self.data_dir, split='test', annotation_level='manufacturer', transform=self.transform())

    def train_dataloader(self):
        loader = DataLoader(self.aircraft_train, batch_size=self.batch_size, num_workers=16)
        return loader

    def val_dataloader(self):
        loader = DataLoader(self.aircraft_val, batch_size=self.batch_size, num_workers=16)
        return loader

    def test_dataloader(self):
        loader = DataLoader(self.aircraft_test, batch_size=self.batch_size, num_workers=16)
        return loader

In [4]:
model = LitCNNLora()
trainer = L.Trainer(
    accelerator="auto",
    devices=1,
    max_epochs=10,
    log_every_n_steps=50
    )
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | model         | MobileNetV2        | 2.9 M 
1 | val_accuracy  | MulticlassAccuracy | 0     
2 | test_accuracy | MulticlassAccuracy | 0     
-----------------------------------------------------
704 K     Trainable params
2.2 M     Non-trainable params
2.9 M     Total params
11.712    Total estimated model params size (MB)


Epoch 0: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 139/139 [00:40<00:00,  3.39it/s, v_num=8, train_loss=3.540]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 139/139 [00:40<00:00,  3.39it/s, v_num=8, train_loss=3.630, val_loss=3.340, val_acc=0.0705]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 139/139 [00:40<00:00,  3.39it/s, v_num=8, train_loss=3.760, val_loss=3.330, val_acc=0.193]
Validation: 0it [00:00, ?it/s]
Epoch 3: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 139/139 [00:41<00:00,  3.39it/s, v_num=8, train_loss=3.680, val_loss=3.280, val_acc=0.197]
Validation: 0it [00:00, ?it/s]
Epoch 4: 100%|██████████████████████████████████████████████████████████

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 139/139 [01:21<00:00,  1.71it/s, v_num=8, train_loss=4.460, val_loss=2.990, val_acc=0.211]


You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/sunil/miniconda3/envs/cnn-lora/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /home/sunil/cnn-lora/lightning_logs/version_8/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | model         | MobileNetV2        | 2.9 M 
1 | val_accuracy  | MulticlassAccuracy | 0     
2 | test_accuracy | MulticlassAccuracy | 0     
----------------------------------

`Trainer.fit` stopped: `max_epochs=10` reached.
